# Early fusion

In [ ]:
import pandas as pd
from torchvision import datasets, transforms
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score

# CIFAR-10 데이터셋 다운로드
transform = transforms.Compose([transforms.ToTensor()])
cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# 데이터 프레임 생성
random_data = pd.DataFrame(np.random.rand(50000, 8), columns=[f'col_{i}' for i in range(8)])

# CIFAR-10 데이터셋에서 이미지의 픽셀 평균값 계산하여 데이터프레임에 추가
pixel_means = []
for i in range(len(cifar_dataset)):
    img, _ = cifar_dataset[i]
    pixel_mean = img.mean().item()  # 이미지의 픽셀 평균값 계산
    pixel_means.append(pixel_mean)

# 데이터프레임에 픽셀 평균값 열 추가
random_data['pixel_mean'] = pixel_means

# 데이터프레임 생성
df = random_data
# 맨 마지막 컬럼에 라벨 추가
labels = []
for i in range(10):
    labels.extend([i] * 5000)

df['label'] = labels

# 학습 데이터셋과 테스트 데이터셋 나누기
train_df = df[:40000]
test_df = df[40000:]

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        features = torch.tensor(self.dataframe.iloc[idx, :-1].values, dtype=torch.float32)
        target = torch.tensor(self.dataframe.iloc[idx, -1], dtype=torch.long)
        return features, target

# 데이터로더 생성
train_dataset = CustomDataset(dataframe=train_df)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(dataframe=test_df)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# 다층 퍼셉트론(MLP) 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(9, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10개의 클래스에 대한 출력 레이어

    def forward(self, x):
        x = torch.flatten(x, 1)  # 입력 데이터를 1차원으로 평평하게 만듦
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 생성
model = MLP()

# 손실 함수 및 최적화 함수 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# F1 스코어 계산 함수 정의
def calculate_f1_score(loader, model):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for features, targets in loader:
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return f1_score(y_true, y_pred, average='weighted')

# 모델 학습 및 테스트
for epoch in range(5):  # 예시로 5 에포크만 학습
    model.train()
    for features, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # 매 에포크마다 테스트 데이터에 대한 F1 점수 출력
    f1 = calculate_f1_score(test_loader, model)
    print(f'Epoch {epoch+1}, Test F1 Score: {f1:.4f}')
